<a href="https://colab.research.google.com/github/jfjoung/AI_For_Chemistry/blob/main/Week_3_Introduction_to_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🎯 **학습 목표:**  
- **딥러닝의 기본 개념**과 **뉴럴 네트워크의 주요 구성 요소**를 이해한다.  
- **기본 뉴럴 네트워크 모델을 설계, 훈련 및 평가하는 과정**을 학습한다.  
- **딥러닝 프레임워크의 사용 방법**을 익히고, 간단한 뉴럴 네트워크를 훈련시켜본다.  
- **딥러닝 모델의 주요 하이퍼파라미터**(예: 학습률, 에포크 수)를 이해하고, 모델 성능을 개선하기 위해 조정해본다.  
- **딥러닝 워크플로우**의 전반적인 이해를 통해, 새로운 문제에 대한 접근 방법을 배운다.  


# 0. 관련 패키지

### PyTorch
Torch 라이브러리를 기반으로 한 PyTorch는 기계 학습 실무자들 사이에서 가장 인기 있는 딥러닝 프레임워크 중 하나입니다. PyTorch를 사용하여 딥러닝 작업을 수행하는 방법을 학습할 예정입니다. 추가 자료는 PyTorch의 [튜토리얼](https://pytorch.org/tutorials/)과 [문서](https://pytorch.org/docs/stable/index.html)를 참고하세요.

### PyTorch Lightning
PyTorch Lightning은 AI 연구자 및 기계 학습 엔지니어를 위한 딥러닝 프레임워크로, 대규모 환경에서도 성능을 포기하지 않고 최대한의 유연성을 제공합니다. 추가 자료는 PyTorch Lightning의 [문서](https://pytorch-lightning.readthedocs.io/en/stable/)를 참고하세요.


In [ ]:
# 필수 라이브러리 설치
! pip install pytorch-lightning torchviz rdkit ogb deepchem torch torchinfo

# 데이터 다운로드
! mkdir data/  # 데이터를 저장할 디렉토리 생성
# ESOL 데이터셋 다운로드
! wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week3/esol.csv -O data/esol.csv
# ESOL 유틸리티 코드 다운로드
! wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week3/esol_utils.py -O esol_utils.py

실험의 재현성을 보장하기 위해 난수 시드를 설정합니다.


In [ ]:
import random
import numpy as np
import torch

# 실험의 재현성을 보장하기 위해 난수 시드를 설정합니다.
torch.manual_seed(0)  # PyTorch의 난수 시드 설정
torch.cuda.manual_seed(0)  # CUDA 연산 시 일관된 결과를 보장하기 위한 난수 시드 설정
np.random.seed(0)  # NumPy의 난수 시드 설정
random.seed(0)  # Python 기본 random 모듈의 난수 시드 설정


# 1. 지도 학습 기반 딥러닝

지난 수업에서 지도 학습(supervised learning)에 대해 익숙해졌을 것입니다. 지도 학습은 레이블이 있는 데이터셋을 사용하여 입력과 출력 간의 관계를 학습하는 머신 러닝의 한 유형입니다.

지금까지 살펴본 모델들은 비교적 간단하며 특정 상황에서는 잘 작동하지만, 때때로 충분하지 않을 수도 있습니다. 이런 경우에는 어떻게 해야 할까요?

<div align="center">
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/deeper_meme.png" width="500"/>
</div>

### 딥러닝
딥러닝(deep learning)은 인공 신경망을 학습시켜 데이터로부터 학습하는 머신 러닝의 하위 분야입니다. 전통적인 머신 러닝 알고리즘은 주로 수작업으로 설계된 특징(feature)과 선형 모델에 의존하는 반면, 딥러닝 알고리즘은 원 데이터(raw data)에서 자동으로 특징을 학습하고 계층적 표현(hierarchy of representations)을 학습할 수 있습니다. 이러한 특성 덕분에 딥러닝 모델은 분자 특성 예측, 반응 예측, 역합성(retrosynthesis) 등과 같은 화학 분야의 다양한 작업에서 최첨단 성능을 달성할 수 있습니다.


#### 데이터: [ESOL 데이터셋](https://pubs.acs.org/doi/10.1021/ci034243x)으로 돌아가기
지난주에 사용했던 ESOL 데이터셋을 다시 활용하겠습니다.  
이전 모델들과의 결과를 비교할 수 있도록 동일한 데이터를 사용합니다.  
데이터 로딩과 전처리를 위해 지난주 코드도 그대로 재사용할 것입니다.


In [ ]:
from esol_utils import load_esol_data

# ESOL 데이터셋을 로드하여 학습, 검증, 테스트 세트로 분할합니다.
(X_train, X_valid, X_test, y_train, y_valid, y_test, scaler) = load_esol_data()


## 2. 신경망 (Neural Networks)

신경망은 인간의 뇌가 작동하는 방식을 모방하도록 설계된 머신 러닝 모델의 한 유형입니다.

<div align="center">
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/nn_image.png" width="500"/>
</div>

\
신경망은 여러 개의 계층(layer)으로 구성된 노드(node)들로 이루어져 있으며, 각 노드는 입력값에 `선형 함수(linear function)`를 적용합니다.  
또한, 비선형 활성 함수(non-linear activation function)를 사용하여 모델에 `비선형성(non-linearity)`을 도입함으로써, 보다 복잡한 패턴을 학습할 수 있도록 합니다.


In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader

# 필요한 라이브러리 및 모듈을 불러옵니다.
# - os: 운영 체제와 상호 작용하기 위한 모듈
# - torch: PyTorch 라이브러리
# - nn: 신경망 모델을 구성하는 PyTorch의 뉴럴 네트워크 모듈
# - F: PyTorch의 함수형 API
# - pytorch_lightning: PyTorch의 고수준 인터페이스로 모델 학습을 간소화
# - DataLoader: PyTorch에서 데이터 배치를 로드하는 유틸리티


## 3. 딥러닝 모델 만들기

요즘에는 딥러닝(DL) 모델을 만드는 것이 비교적 쉬워졌습니다. 특히, [PyTorch Lightning](https://pytorch-lightning.readthedocs.io/en/stable/index.html)과 같은 라이브러리 덕분에 많은 작업이 자동화되었지만, 여전히 모델에 대한 세부적인 제어가 가능합니다.

PyTorch Lightning을 사용하려면 먼저 **클래스(class)**에 대해 알아야 합니다.

> 클래스는 특정 속성과 동작을 가진 객체를 생성하기 위한 템플릿 또는 지침 세트라고 생각할 수 있습니다.  
> 클래스에서 생성된 객체를 인스턴스(instance)라고 합니다.

\
예를 들어, 개(Dog)를 표현하는 프로그램을 만든다고 가정해봅시다.

```python
class Dog:
    def __init__(self, name, color):
        self.name = name
        self.color = color
        
    def say_your_name(self):
        print(f"내 이름은 {self.name}입니다.")
```

이 예제에서 개는 `name`과 `color`라는 두 개의 속성을 가집니다. 또한, `say_your_name`이라는 메서드 (method)를 가지고 있습니다.

이제 원하는 만큼 개 객체를 만들 수 있습니다.

```python
lassie = Dog(name="Lassie", color="White")
pluto = Dog(name="Pluto", color="Yellow")
```

그리고 객체의 메서드를 호출하여 사용할 수 있습니다.
```python
pluto.say_your_name()   # 출력: "내 이름은 Pluto입니다."
```

<font color="#4caf50" size=4> 이제 신경망(Neural Network) 클래스를 정의해봅시다. </font>

각 부분의 역할:

- **`__init__`**: 모델의 아키텍처를 정의하는 부분입니다.  
  다양한 레이어(layer, 모델의 구성 요소)를 추가할 수 있으며, 이곳에서 모델 구조를 결정합니다.

- **`training_step`**: 모델 학습 과정에서 사용되는 메서드로, 옵티마이저를 이용해 모델의 파라미터를 업데이트합니다.

- **`configure_optimizers`**: 옵티마이저를 설정하는 메서드입니다.  
  여기에서 학습률(learning rate)과 사용할 옵티마이저를 정의합니다.

- **`forward`**: 입력값이 주어졌을 때 모델이 수행해야 할 연산을 지정합니다.


In [ ]:
class NeuralNetwork(pl.LightningModule):
    def __init__(self, input_sz, hidden_sz, train_data, valid_data, test_data, batch_size=254, lr=1e-3):
        """
        PyTorch Lightning 기반의 신경망 모델 클래스
        - input_sz: 입력 크기 (특성 수)
        - hidden_sz: 은닉층 뉴런 수
        - train_data, valid_data, test_data: 학습, 검증, 테스트 데이터셋
        - batch_size: 미니배치 크기
        - lr: 학습률 (learning rate)
        """
        super().__init__()  # 부모 클래스(pl.LightningModule)의 초기화 메서드 호출
        self.lr = lr  # 학습률(learning rate) 설정
        self.train_data = train_data  # 학습 데이터 저장
        self.valid_data = valid_data  # 검증 데이터 저장
        self.test_data = test_data  # 테스트 데이터 저장
        self.batch_size = batch_size  # 배치 크기 설정

        # 학습 및 검증 손실값을 저장할 리스트 추가
        self.train_losses = []  # 학습 손실 저장 리스트
        self.val_losses = []  # 검증 손실 저장 리스트
        self.epoch_train_losses = []  # Train loss를 Epoch마다 평균 저장할 리스트

        # 신경망 모델 정의
        self.model = nn.Sequential(
            nn.Linear(input_sz, hidden_sz),  # 입력층 -> 은닉층 변환
            nn.ReLU(),  # 활성화 함수 ReLU 적용
            nn.Linear(hidden_sz, hidden_sz),  # 은닉층 추가
            nn.ReLU(),  # 활성화 함수 ReLU 적용
            nn.Linear(hidden_sz, 1)  # 출력층 (연속적인 값 예측)
        )

    def training_step(self, batch, batch_idx):
        """
        모델 학습 단계 정의
        - 배치 데이터에서 입력(x)과 정답(y)을 분리
        - 모델을 통해 예측값 계산
        - 손실(MSE) 계산 후 저장
        """
        x, y = batch  # 배치 데이터에서 입력(x)과 정답(y) 분리
        z = self.model(x)  # 모델을 이용해 예측값 계산
        loss = F.mse_loss(z, y)  # 평균 제곱 오차(MSE) 손실 계산

        # 손실 저장
        self.train_losses.append(loss.item())  # 리스트에 손실값 추가

        self.log("Train loss", loss)  # PyTorch Lightning 로깅
        return loss  # 손실 반환

    def validation_step(self, batch, batch_idx):
        """
        검증 단계 정의 (에포크 종료 시 실행)
        - 배치 데이터에서 입력(x)과 정답(y)을 분리
        - 모델을 통해 예측값 계산
        - 손실(MSE) 계산 후 저장
        """
        x, y = batch  # 배치 데이터에서 입력(x)과 정답(y) 분리
        z = self.model(x)  # 모델을 이용해 예측값 계산
        loss = F.mse_loss(z, y)  # 평균 제곱 오차(MSE) 손실 계산

        # 검증 손실 저장
        self.val_losses.append(loss.item())  # 리스트에 손실값 추가

        self.log("Valid MSE", loss)  # PyTorch Lightning 로깅

    def on_train_epoch_end(self):
        # 에포크가 끝날 때 Train loss를 평균 내어 저장
        epoch_avg_loss = sum(self.train_losses) / len(self.train_losses)
        self.epoch_train_losses.append(epoch_avg_loss)
        self.train_losses = []  # 배치 단위 loss 리스트 초기화

    def test_step(self, batch, batch_idx):
        """
        테스트 단계 정의
        - 테스트 데이터에 대한 성능 평가
        """
        x, y = batch
        z = self.model(x)
        loss = F.mse_loss(z, y)
        self.log("Test MSE", loss)

    def configure_optimizers(self):
        """
        옵티마이저(Adam) 설정
        - 모델의 모든 학습 가능한 파라미터를 최적화
        """
        optimizer = torch.optim.Adam(
            self.parameters(),  # 모델의 학습 가능한 모든 파라미터 전달
            lr=self.lr  # 학습률 설정
        )
        return optimizer  # 옵티마이저 반환

    def forward(self, x):
        """
        모델 순전파(forward pass) 정의
        """
        return self.model(x).flatten()  # 신경망을 통과한 결과를 1차원으로 변환하여 반환

    def train_dataloader(self):
        """
        학습 데이터 로더 생성
        - 미니배치를 섞어서 학습 진행
        """
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        """
        검증 데이터 로더 생성
        - 배치 순서 유지
        """
        return DataLoader(self.valid_data, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        """
        테스트 데이터 로더 생성
        - 배치 순서 유지
        """
        return DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False)


### 데이터셋(Dataset) 클래스

Lightning을 사용하려면 `Dataset` 클래스를 생성해야 합니다.  
조금 복잡해 보일 수 있지만, 더 복잡한 상황에서도 유연성을 제공하므로 겁먹지 않아도 됩니다! 😉


In [ ]:
from torch.utils.data import Dataset  # PyTorch의 Dataset 클래스를 가져옴

class ESOLDataset(Dataset):
    def __init__(self, X, y):
        """
        ESOL 데이터셋을 PyTorch Dataset 형태로 변환하는 클래스
        :param X: 입력 데이터 (특징 벡터)
        :param y: 타겟 데이터 (정답 라벨)
        """
        self.X = X  # 입력 데이터 저장
        self.y = y  # 타겟 데이터 저장

    def __len__(self):
        """
        데이터셋의 샘플 개수를 반환
        :return: 데이터셋의 전체 크기
        """
        return self.X.shape[0]

    def __getitem__(self, idx):
        """
        인덱스를 기반으로 데이터 샘플을 가져오는 메서드
        :param idx: 샘플의 인덱스
        :return: 변환된 입력 데이터(X_), 타겟 데이터(y_)
        """
        if torch.is_tensor(idx):  # 인덱스가 텐서 형태일 경우 리스트로 변환
            idx = idx.tolist()

        X_ = torch.as_tensor(self.X[idx].astype(np.float32))  # 입력 데이터를 PyTorch 텐서로 변환
        y_ = torch.as_tensor(self.y[idx].astype(np.float32).reshape(-1))  # 타겟 데이터를 PyTorch 텐서로 변환 및 차원 조정

        return X_, y_  # 변환된 데이터를 반환

# 데이터셋 인스턴스 생성
train_data = ESOLDataset(X_train, y_train)  # 학습 데이터
valid_data = ESOLDataset(X_valid, y_valid)  # 검증 데이터
test_data = ESOLDataset(X_test, y_test)  # 테스트 데이터


In [ ]:
# 신경망 모델 인스턴스 생성
nn_model = NeuralNetwork(
    input_sz=X_train.shape[1],  # 입력 크기 설정
    hidden_sz=128,  # 은닉층 크기 설정
    train_data=train_data,  # 학습 데이터
    valid_data=valid_data,  # 검증 데이터
    test_data=test_data,  # 테스트 데이터
    lr=1e-3,  # 학습률 설정
    batch_size=256  # 배치 크기 설정
)

# 모델 구조 확인
from torchinfo import summary

summary(nn_model, input_size=(256, X_train.shape[1]))  # 배치 크기 포함



In [ ]:
import torch
from torchviz import make_dot

# 모델을 실행할 장치 설정 (GPU 사용 가능하면 GPU로, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nn_model.to(device)  # 모델을 해당 장치로 이동

# 더미 입력 데이터 생성 (X_train의 크기와 동일하게 설정)
dummy_input = torch.randn(1, X_train.shape[1]).to(device)

# 모델의 연산 그래프 생성
output = nn_model(dummy_input)
graph = make_dot(output, params=dict(nn_model.named_parameters()))

# 그래프 시각화 및 저장
graph.render("model_structure", format="png", cleanup=True)

# 생성된 'model_structure.png' 파일을 확인하세요.


In [ ]:
# 모델 학습
# PyTorch Lightning Trainer 설정
trainer = pl.Trainer(max_epochs=200)

# 모델 학습 시작
trainer.fit(model=nn_model)

# 테스트 실행
results = trainer.test(ckpt_path="best")

In [ ]:
import matplotlib.pyplot as plt

# Loss 시각화
plt.figure(figsize=(10, 5))
plt.plot(nn_model.epoch_train_losses, label="Train Loss")  # 학습 loss 그래프
plt.plot(nn_model.val_losses, label="Validation Loss")  # 검증 loss 그래프
plt.xlabel("Epochs")  # X축: 에포크
plt.ylabel("Loss")  # Y축:  Loss
plt.title("Training and Validation Loss Over Epochs")  # 제목 설정
plt.legend()  # 범례 추가
plt.grid()  # 격자 추가
plt.show()  # 그래프 출력

In [ ]:
  # 테스트 RMSE 계산
test_mse = results[0]["Test MSE"]  # 테스트 데이터에서의 MSE 값 가져오기
test_rmse = test_mse ** 0.5  # RMSE(Root Mean Squared Error) 계산

# 결과 출력
print(f"\nANN model performance: RMSE on test set = {test_rmse:.4f}\n")


# 연습 문제:

하이퍼파라미터를 조정해 보면서 결과가 어떻게 변하는지 확인해보세요.

`hidden_sz`, `batch_sz`, `max_epochs`, `lr` 등의 값을 변경해 볼 수 있습니다.  
또는 신경망의 구조 자체를 수정하여, 레이어 개수를 변경하거나 활성화 함수를 바꿔보는 것도 좋은 방법입니다.


하이퍼파라미터를 바꾸어가며 만든 모델 들 중, 무엇이 가장 좋습니까?
가장 좋은 이유는 무엇일까요?

Test set에 대하여 RMSE가 가장 작은 모델로 여러 분자들의 용해도를 예측해봅시다.

In [ ]:
from deepchem.feat import RDKitDescriptors
from rdkit import Chem
from rdkit.Chem import Descriptors

def smiles_to_descriptors(smiles, scaler):
    """
    SMILES 문자열을 RDKit Descriptors로 변환하는 함수
    :param smiles: 변환할 SMILES 문자열
    :param scaler: 학습 데이터에서 학습된 MinMaxScaler 객체
    :return: 변환된 특징 벡터 (PyTorch 텐서)
    """
    featurizer = RDKitDescriptors()  # RDKit 분자 기술자(Descriptors) 생성
    features = featurizer.featurize([smiles])  # 단일 SMILES를 특징 벡터로 변환
    features = np.array(features, dtype=np.float32)  # NumPy 배열 변환

    # 학습 데이터와 일관성을 유지하기 위해 동일한 스케일링 적용
    features = features[:, ~np.isnan(features).any(axis=0)]  # NaN 값 제거
    features = scaler.transform(features)  # 학습된 스케일러를 적용하여 변환
    return torch.tensor(features).float()  # PyTorch 텐서로 변환

def predict_solubility(model, smiles, scaler):
    """
    학습된 모델을 사용하여 SMILES의 용해도를 예측하는 함수
    :param model: 학습된 신경망 모델
    :param smiles: 용해도를 예측할 SMILES 문자열
    :param scaler: 데이터 스케일링을 위한 MinMaxScaler 객체
    :return: 예측된 용해도 값
    """
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():  # 그래디언트 계산 비활성화
        X = smiles_to_descriptors(smiles, scaler)  # SMILES를 특징 벡터로 변환
        prediction = model(X).item()  # 모델을 통해 예측 수행
    return prediction


def calculate_molecular_weight(smiles):
    """
    주어진 SMILES 문자열의 1 mol당 g(분자량, Molecular Weight)를 계산하는 함수
    :param smiles: 분자 구조를 나타내는 SMILES 문자열
    :return: 분자량 (g/mol)
    """
    mol = Chem.MolFromSmiles(smiles)  # SMILES 문자열을 RDKit 분자 객체로 변환
    if mol is None:
        raise ValueError("유효하지 않은 SMILES 문자열입니다.")

    mw = Descriptors.MolWt(mol)  # RDKit을 사용하여 분자량 계산
    return mw

In [ ]:
trained_model = nn_model  # 학습된 모델 사용
test_smiles = "O=C(C)Oc1ccccc1C(=O)O"  # 예측할 SMILES (예: 아스피린)
predicted_log_solubility = predict_solubility(trained_model, test_smiles, scaler)
predicted_solubility_mol_L = 10 ** predicted_log_solubility

print(f"SMILES: {test_smiles}")
print(f"Predicted log Solubility: {predicted_log_solubility:.4f} log(mol/L)")
print(f"Predicted Solubility: {predicted_solubility_mol_L:.6f} mol/L")
print(f"Predicted Solubility: {predicted_solubility_mol_L*calculate_molecular_weight(test_smiles):.6f} g/L")

# 아스피린의 용해도는 3 g/L 입니다.